In [ ]:
!pip install open3d

In [2]:
import numpy as np
import open3d as o3d
import plotly.graph_objects as go
from google.colab import drive
import cv2
from google.colab.patches import cv2_imshow
from PIL import Image
import math
import itertools
import copy
import matplotlib.pyplot as plt
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
def draw_geometries(geometries):
    graph_objects = []

    for geometry in geometries:
        geometry_type = geometry.get_geometry_type()
        
        if geometry_type == o3d.geometry.Geometry.Type.PointCloud:
            points = np.asarray(geometry.points)
            colors = None
            if geometry.has_colors():
                colors = np.asarray(geometry.colors)
            elif geometry.has_normals():
                colors = (0.5, 0.5, 0.5) + np.asarray(geometry.normals) * 0.5
            else:
                geometry.paint_uniform_color((1.0, 0.0, 0.0))
                colors = np.asarray(geometry.colors)

            scatter_3d = go.Scatter3d(x=points[:,0], y=points[:,1], z=points[:,2], mode='markers', marker=dict(size=1, color=colors))
            graph_objects.append(scatter_3d)

        if geometry_type == o3d.geometry.Geometry.Type.TriangleMesh:
            triangles = np.asarray(geometry.triangles)
            vertices = np.asarray(geometry.vertices)
            colors = None
            if geometry.has_triangle_normals():
                colors = (0.5, 0.5, 0.5) + np.asarray(geometry.triangle_normals) * 0.5
                colors = tuple(map(tuple, colors))
            else:
                colors = (1.0, 0.0, 0.0)
            
            mesh_3d = go.Mesh3d(x=vertices[:,0], y=vertices[:,1], z=vertices[:,2], i=triangles[:,0], j=triangles[:,1], k=triangles[:,2], facecolor=colors, opacity=0.50)
            graph_objects.append(mesh_3d)
        
    fig = go.Figure(
        data=graph_objects,
        layout=dict(
            scene=dict(
                xaxis=dict(visible=False),
                yaxis=dict(visible=False),
                zaxis=dict(visible=False)
            )
        )
    )
    fig.show()
o3d.visualization.draw_geometries = draw_geometries # replace function

In [4]:
%cd /content/drive/MyDrive/3DDP23_HW3/
!ls

/content/drive/MyDrive/3DDP23_HW3
office.ply  pc_feature.ipynb  pc_reconstruction.ipynb  pc_registration.ipynb


In [7]:
#  여기서부터 작성하세요

# 학번: 2021103751 
# 이름: 정은성

demo_icp_pcds = o3d.data.OfficePointClouds()
# demo_icp_pcds = o3d.data.DemoICPPointClouds()

# global 값 init
g_cloud = o3d.geometry.PointCloud()
g_Desc = o3d.pipelines.registration.Feature()


# target = o3d.io.read_point_cloud(demo_icp_pcds.paths[22])
prev_source = o3d.io.read_point_cloud(demo_icp_pcds.paths[0])

def pose_eval(source, target, T, thresold):
  evaluation = o3d.pipelines.registration.evaluate_registration(source, target, thresold, T)
  return evaluation

for i in range(1, 29):
  # 이전 것과 이어 붙일 source
  now_source = o3d.io.read_point_cloud(demo_icp_pcds.paths[i])
  
  # source를 0.02인 작은 범위로 down sampling 연산
  voxel_size = 0.02
  prev_source_big = prev_source.voxel_down_sample(voxel_size=voxel_size)
  now_source_big = now_source.voxel_down_sample(voxel_size=voxel_size)


  # source를 0.04로 큰 범위로 down sampling 연산
  voxel_size = 0.04
  prev_source = prev_source.voxel_down_sample(voxel_size=voxel_size)
  now_source = now_source.voxel_down_sample(voxel_size=voxel_size)
  

  radius_normal = voxel_size * 2
  radius_feature = voxel_size * 3


  prev_source.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))
  now_source.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

  source_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        prev_source, o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
 
  target_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        now_source, o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
  
  # RANSAC
  distance_threshold = voxel_size 

  result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        prev_source, now_source, source_fpfh, target_fpfh, True, distance_threshold,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
        3, [o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.3),
        o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)], 
        o3d.pipelines.registration.RANSACConvergenceCriteria(10000000, 0.99))
  

  T = result.transformation


  source2 = copy.deepcopy(prev_source_big).transform(T)
  target2 = copy.deepcopy(now_source_big)

  refinement_result =o3d.pipelines.registration.registration_icp(prev_source, now_source, distance_threshold, T,
            o3d.pipelines.registration.TransformationEstimationPointToPlane())

  T_ref = refinement_result.transformation

  source2 = copy.deepcopy(prev_source_big).transform(T_ref)
  target2 = copy.deepcopy(now_source_big)
  
  prev_source = source2 + target2
  g_cloud = prev_source
  

o3d.io.write_point_cloud("office.ply", g_cloud)

True